#  Where (not) to eat in Chicago? Project milestone 2


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
data_folder = "data"

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
import json
import folium
from folium import plugins
from folium.plugins import MarkerCluster, Search

import shapely
from shapely.geometry import shape, Point

In [ ]:
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

We start by reading our data

In [ ]:
inspections = pd.read_csv(os.path.join(data_folder, "food-inspections.csv"))

We can now look into the fetures and types of our dataset through a sample :

In [ ]:
inspections.head()

In [ ]:
inspections.columns

First thing we notice is the strange presence of the columns Historical Wards 2003-2015, Zip Codes, Community Areas, Census Tracts and Wards. These columns all have only NaN values, so we are dropping them:

In [ ]:
inspections.drop(columns=['Historical Wards 2003-2015', 'Zip Codes',
       'Community Areas', 'Census Tracts', 'Wards'],inplace=True)

Let's see what are the possible categories of our facilities:

In [ ]:
inspections["Facility Type"].unique()

We can notice very different facility types; as we want to analyze only restaurants we will keep only entries where "Restaurant" is the Facility Type. Let's see if we have any entries with missing Facility Type:

In [ ]:
print("There are {0} missing values for Facility Type in the dataset".format(len(inspections[inspections["Facility Type"].isna()])))

For now we will get rid of those rows as they are of no use to us:

In [ ]:
inspections = inspections.dropna(subset=["Facility Type"])

We only want to analyze entries which correspond to restaurants so we will keep only those rows

In [ ]:
# Keep only restaurants for the rest of the analysis
inspections = inspections[inspections["Facility Type"].str.contains("restaurant", case=False)]

In [ ]:
print(f"We have {len(inspections)} rows in our dataset, each row corresponds to one inspection")

Since we are going to compare inspection's results over time, let's look at the time range we have for the inspections.

In [ ]:
# Cast Inspection Date to Datetime type
inspections['Inspection Date'] = pd.to_datetime(inspections['Inspection Date'])

In [ ]:
print('The first inspection in the dataset happened in {0}, and the last one happened was on {1}'. 
      format(inspections['Inspection Date'].min().strftime("%d.%m.%Y"), inspections['Inspection Date'].max().strftime("%d.%m.%Y.")))

## Data preprocessing

We will now proceed with preparing the dataset for the analysis. Below we take the following preprocessing steps:
1. Making Licence # column a unique identifier of restaurants in the dataset.
2. Fixing the zip codes by completing the missing values based on the values of Longitude and Latitude.
3. Fixing the City column by completing the missing values based on the zip code.
4. Standardizing the Inspection Type values by cleaning various namings and reducing the number of categories.

### Making unique licenses

The original dataset we have is inspections-oriented, but our analysis will be mainly focused on restaurants. That is why we need a feature which will help us uniquely identify each restaurant.

In [ ]:
print(f"Using DBA Name we can see there are {inspections['DBA Name'].nunique()} distinct restaurant names")

In [ ]:
print(f"Using restaurant Licenses we can see there are {inspections['License #'].nunique()} distinct restaurants")

We have probably many non unique names due to restaurants such as McDonalds and other large food-serving chains. We want to make the license number our primary key for restaurant identification as we assume different restaurants have different license numbers. First, let's check if there are any missing values in this column:

In [ ]:
print(f"There are {inspections['License #'].isna().sum()} restaurant that have NaN licence number")

We noticed however that there are some restaurant with licence numbers equal to 0, let's see how many there are:

In [ ]:

print(f"There are {len(inspections[inspections['License #'] == 0.0])} restaurant that have zero licence number")

As we can see, there is no column that can help us uniquely identify a restaurant. The closest match is License # feature, which has some restaurants with 0 assigned as their license number. We decided to add fake licence numbers and use it as a restaurant identifier later on.

In [ ]:
# Get entries where License number is missing
zero_license = inspections[inspections["License #"] == 0.0]

In [ ]:
zero_license.nunique()

To be able to identify restaurants with license number equal to 0, we will assume that no two of such restaurants share the same location and the same name. Using this assumption, we can create artificial license numbers for restaurants with missing license numbers:

In [ ]:
# Get the maximum license number that exists in the database and use it as starting point for the newly generated license numbers
start_id = int(inspections["License #"].max() + 1)
missing_licenses = inspections[(inspections["License #"] == 0)][["DBA Name","Location"]].copy().drop_duplicates()
missing_licenses["new License"] = [i for i in range(start_id, start_id+len(missing_licenses.drop_duplicates()))]

In [ ]:
# Merge dataset
inspections = inspections.merge(missing_licenses, on=["DBA Name","Location"], how='left')

In [ ]:
# Populate missing licence numbers with newly generated license numbers
inspections["License #"] = inspections["License #"].apply(lambda x: np.nan if x == 0 else x)
inspections["License #"] = inspections["License #"].fillna(value = inspections["new License"])
inspections.drop(columns=["new License"], inplace=True)

In [ ]:
print("There are {0} missing license numbers in our dataset".format(inspections[inspections["License #"] == 0].size))

In [ ]:
# Cast License number from float to int
inspections["License #"] = inspections["License #"].apply(lambda x : int(x))

### Completing zip codes based on geographical coordinates

The big part of our analysis will be based on neighborhoods, therefore we will hugely rely on the Zip column.

We want to populate the missing Zip values based on the Latitude and Longitude. In order to do that, we must have those two features for all the restaurants in the dataset. That is why we decided to drop all entries where one of those features is missing.

In [ ]:
inspections = inspections[~((inspections['Longitude'].isna()) | (inspections['Latitude'].isna()))]

In [ ]:
print('There are {0} missing values for Zip column'.format(inspections[inspections['Zip'].isna()].shape[0]))
print('There are {0} missing values for City column'.format(inspections[inspections['City'].isna()].shape[0]))

In [ ]:
restaurants_zip_na = inspections[inspections['Zip'].isna()]

To populate Zip values based on coordinates, we use [shapely](https://pypi.org/project/Shapely/) - Python package for manipulation and analysis of planar geometric objects.

In [ ]:
# Function that creates points from Latitude and Longitude
def create_points(df):
    coords = list(zip(df['Longitude'], df['Latitude']))
    res = []
    for coord in coords:
        res.append(shapely.geometry.Point(coord))
    return res

In [ ]:
# Create list of points for which we want to get Zip code
points = create_points(restaurants_zip_na)

In [ ]:
# Method which checks whether the points are in area described in geojson file and returns data with zip value for found points
def populate_missing_zip(points, geojson_filename):
    # load GeoJSON file containing sectors
    state_geo_path = r'{0}'.format(geojson_filename)
    geo_json_data = json.load(open(state_geo_path))

    zip_found = []
    # check each polygon to see if it contains the point
    for feature in geo_json_data['features']:
        polygon = shapely.geometry.shape(feature['geometry'])
        for point in points: 
            if polygon.contains(point):
                point_complete = {'Longitude':point.x, 'Latitude':point.y, 'Zip':feature.get('properties', {}).get('zip')}
                zip_found.append(point_complete)
    return zip_found

In [ ]:
# Find missing Zip values
zip_found = populate_missing_zip(points, os.path.join(data_folder, 'chicago-zip.json'))
print('Total {0} point found matching Chicago sectors.'.format(len(zip_found)))

In [ ]:
zip_found = pd.DataFrame(zip_found)
zip_found.head()

Now, we have to merge those results with the original dataset.

In [ ]:
# Before merging, drop duplicate points
zip_found.drop_duplicates(inplace=True)

In [ ]:
inspections = inspections.merge(zip_found,on=['Latitude','Longitude'], how='left',suffixes=('', '_notnull'))
inspections.Zip.fillna(value=inspections.Zip_notnull, inplace=True)
inspections.drop(columns=["Zip_notnull"], inplace=True)

In [ ]:
print('There are {0} missing Zip left in the restaurant dataset.'.format(inspections[inspections.Zip.isnull()].shape[0]))

In [ ]:
# Change type of Zip feature from float to string
inspections['Zip']=inspections['Zip'].apply(lambda x: str(int(x)))

### Completing City column based on zip codes

We will also fix the City column. Now we can use the Zip column to fill in the missing information about the city.

In [ ]:
# Function that returns all Chicago Zips frem geojson file
def create_chicago_zip_list():
    state_geo_path = os.path.join(data_folder, "chicago-zip.json")
    geo_json_data = json.load(open(state_geo_path))

    zips = []
   
    for feature in geo_json_data['features']:
        zips.append(str(feature.get('properties', {}).get('zip')))
    return set(zips)

In [ ]:
# Get list of all Zip codes in Chicago
chicago_zip = create_chicago_zip_list()

In [ ]:
# Check if there is any restaurant not in Chicago
not_in_chicago = len(inspections[inspections.City.isna() & (~inspections.Zip.isin(chicago_zip))])
print('There are {0} Zip values which are not in Chicago.'.format(not_in_chicago))

In [ ]:
# Replace all City missing values with Chicago
inspections.City.fillna(value='Chicago', inplace=True)

In [ ]:
print('There are {0} missing City values left in the restaurant dataset.'.format(inspections[inspections.City.isnull()].shape[0]))

In [ ]:
inspections.City.unique()

In [ ]:
# Change all values for City column to be Chicago
inspections.City = 'Chicago'
inspections.City.unique()

### Standardizing/cleaning Inspection Type values

In [ ]:
inspections['Inspection Type'].unique()

Looking at `Inspection Type` values we can see that they need some cleaning. According to the [document describing the dataset](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF) we should have following inspection types:
 - **canvass**: regular inspections with frequency depending on establishment risk,
 - **complaint**: in a response to filed complaint,
 - **license**: when obtaining a license, as a requirement of launching the establishment (should be once for most or more if they failed, this might be a separate study case),
 - **suspect food poisoning**: specific type of *complaint* when someone reports getting ill after eating from there
 - **task-force**: for bars and taverns.

Also, as the linked document states, re-inspections can be done for most of the types and are indicated in the name of inspection type.

First, we replace NaN values in Inspection Type with "Unknown":

In [ ]:
inspections.fillna(value={'Inspection Type': 'Unknown'}, inplace=True)

Now, since we may want to use the indication whether a particular inspection was a re-inspection or not, we add a separate column that will indicate that:

In [ ]:
reinspection_pattern = 're-inspec|reinspec|re inspec'
inspections['Re-inspection'] = inspections['Inspection Type'].str.lower().str.contains(reinspection_pattern, regex=True)

Now, we proceed with making the names for given types of inspections uniform:

In [ ]:
inspection_types = inspections['Inspection Type'].unique().astype(str)
inspection_types_lower = np.char.lower(inspection_types)

In [ ]:
# Replaces values in Inspection Type for records with keywords found in them with the specified replacement value
def standardize_by_finding_keyword(keywords, replacement):
  to_replace = np.array([])
  for keyword in keywords:
    to_replace = np.append(to_replace, inspection_types[np.char.find(inspection_types_lower, keyword) != -1])
  inspections['Inspection Type'] = inspections['Inspection Type'].replace(to_replace, value=replacement)

In [ ]:
standardize_by_finding_keyword(['canvas'], 'Canvass')
standardize_by_finding_keyword(['complain'], 'Complaint')
standardize_by_finding_keyword(['license'], 'License')
standardize_by_finding_keyword(['task', 'liquor'], 'Task Force')

In [ ]:
# Suspected Food Poisoning replacements
sfp_values = inspections['Inspection Type'].str.lower().str.contains('food|sfp', regex=True)
inspections.loc[sfp_values, 'Inspection Type'] = 'Suspected Food Poisoning'
standardize_by_finding_keyword(['sick'], 'Suspected Food Poisoning')

In [ ]:
inspections.groupby(by='Inspection Type')['Inspection ID'].count()

There are still a lot of values that appear only once in the entire dataset. There are also ones that could be merged into single categories (e.g. "no entry", "out of business", "recent inspection"). We also decide to leave the categories with significant amount of records, such as "Consultation". The ones we decide to drop will be reclassified under "Unknown" category.

In [ ]:
# Replaces records with keyword found in category name to be classified in a given target category
def merge_categories(keyword, target_category):
  categories_containing_keyword = inspections['Inspection Type'].str.lower().str.contains(keyword)
  inspections.loc[categories_containing_keyword, 'Inspection Type'] = target_category  

In [ ]:
merge_categories('recent inspection', 'Recent Inspection')
merge_categories('out of business', 'Out of Business')
merge_categories('no entry', 'No Entry')

In [ ]:
known_list = ['License', 'Canvass', 'Complaint', 'Consultation', 'No Entry', 'Out of Business', 'Recent Inspection', 'Suspected Food Poisoning', 'Tag Removal', 'Task Force']
# Classify the rest as unknown
inspections.loc[~inspections['Inspection Type'].isin(known_list), 'Inspection Type'] = 'Unknown'

In [ ]:
inspections.groupby(by='Inspection Type')['Inspection ID'].count()

Additionally, let's propagate "Out of Business" indication to "Results" column where it should be indicated:

In [ ]:
out_of_business = inspections[inspections['Inspection Type'] == 'Out of Business'].index
inspections.loc[out_of_business, 'Results'] = 'Out of Business'

### Creating Community and District Column

Chicago is divided into 77 Community Areas that can also be grouped into 9 Districts. Even though we have Zip codes for each restaurant, the role of Zip codes in real world is mainly related to post office and delivery services. People who live in Chicago and visit Chicago are, however, using Community Areas and districts for orientation. We decided to enrich the dataset with this information. We are using the chicago-community-areas geojson file for getting boundaries of community areas, and shapely library to check for each coordinate in which community area it belongs.

In [ ]:
# Extract unique Latitude and Longitude from inspections
unique_coords = inspections.copy()[['Latitude','Longitude']].drop_duplicates()

# Create shapely points
points = create_points(unique_coords)

In [ ]:
# Method which checks whether the points are in area described in geojson file and returns data with community area name for found points
def populate_area(points, geojson_filename):
    # load GeoJSON file containing sectors
    state_geo_path = r'{0}'.format(geojson_filename)
    with open(state_geo_path) as file:
        geo_json_data = json.load(file)

        area_found = []
        # check each polygon to see if it contains the point
        for feature in geo_json_data['features']:
            polygon = shapely.geometry.shape(feature['geometry'])
            for point in points: 
                if polygon.contains(point):
                    point_complete = {'Longitude':point.x, 'Latitude':point.y, 'Community':feature.get('properties', {}).get('community')}
                    area_found.append(point_complete)
        return area_found

In [ ]:
area_found = populate_area(points, os.path.join(data_folder, 'chicago-community-areas.json'))
print('Total {0} point found matching Chicago sectors.'.format(len(area_found)))

In [ ]:
area_found = pd.DataFrame(area_found)
inspections = inspections.merge(area_found,on=['Latitude','Longitude'], how='left')
print('There are {0} missing Community left in the restaurant dataset.'.format(inspections[inspections.Community.isnull()].shape[0]))

Since we have some missing values after populating the Community values from geojson, we will try to fix them manually. Let's see if there is some pattern in the missing data.

In [ ]:
inspections[inspections.Community.isna()].Zip.unique()

It seems that the only missing data are from two Zip codes. By doing quick google search and looking at maps, we can see that 60666 refers to O'Hare International Airport and 60611 is Near North Side.

In [ ]:
# Populate missing values
inspections.loc[((inspections.Community.isna()) & (inspections.Zip == '60666')), 'Community'] = "O'HARE INTERNATIONAL AIRPORT"
inspections.loc[((inspections.Community.isna()) & (inspections.Zip == '60611')), 'Community'] = "NEAR NORTH SIDE"

print('There are {0} missing Community left in the restaurant dataset.'.format(inspections[inspections.Community.isna()].shape[0]))

In [ ]:
inspections.Community.unique()

We are also going to get the information about Districts for each restaurant. For that we created a mapping between Communities and Districts, which we load from csv file.

In [ ]:
chicago_communities = pd.read_excel(os.path.join(data_folder, 'Chicago-Communities.xlsx'))
chicago_communities.head()

In [ ]:
chicago_communities.Community = chicago_communities.Community.apply(lambda x: str.upper(x).strip())
#chicago_communities.District = chicago_communities.District.apply(lambda x: string.capwords(x).strip())

In [ ]:
inspections =inspections.merge(chicago_communities, on='Community', how='left')
print('There are {0} missing District left in the restaurant dataset.'.format(inspections[inspections.District.isna()].shape[0]))

In [ ]:
inspections.head()

##### INSTRUCTION FOR MAPS WITH GROUP BY COMMUNITIES

In [ ]:
inspections.Community.nunique()

Apparently we have a community which does not have any inspection. (wierd)? So when you plot chrolophlet with chicago-community-areas.json you have black space there. There are 2 options:

1. use **chicago-community-areas-1.json** where I removed that community area so it is not plotted (like below)
2. manualy add 0 entraince for that community area before you plot (but be careful, because having 0 failures for instance could be misinterpreted as great neighborhood, while in reality there were no inspections)

The only difference between previous folium maps by zip is grouping by community and key_on = 'feature.properties.community'

In [ ]:
#### MAP EXAMPLE
group_by_area = inspections.copy().groupby('Community').count()
group_by_area.reset_index(level=0,inplace=True)
group_by_area['Community'] = group_by_area['Community'].apply(lambda x: str(x))

state_geo_path = os.path.join(data_folder, "chicago-community-areas.json")
geo_json_data = json.load(open(state_geo_path))

map_safe_danger = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
map_safe_danger.choropleth(geo_data=geo_json_data, data=group_by_area,
             columns=['Community', 'Results'],
             key_on='feature.properties.community',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Count inspections per area')
#map_safe_danger
map_safe_danger.save(os.path.join(data_folder,'random.html'))

In [ ]:
group_by_area

## Descriptive analysis

In this part we prepare some basic plots to get more visual insight in our dataset.

In [ ]:
sns.set(rc={'figure.figsize':(10,6)})
sns.set_style("whitegrid")

We start with number of inspections per year to see if we can observe any trend in that matter

In [ ]:
data = inspections.groupby(inspections["Inspection Date"].dt.year)["Inspection ID"].count().to_frame().reset_index()
data = data.rename(columns={"Inspection ID": "Number of inspections"})
sns.barplot(data["Inspection Date"], data["Number of inspections"]).set_title("Number of inspections per year")

Next we want to look into inspections results each year

In [ ]:
data = inspections.groupby([inspections["Inspection Date"].dt.year, inspections["Results"]])["Inspection ID"].count().to_frame().reset_index()
data = data.rename(columns={"Inspection ID": "Number of inspections"})
sns.barplot(data["Inspection Date"], data["Number of inspections"], data["Results"]).set_title("Number of inspections by result per year")

It seems that Chicago sanitary got more strict as we can see in 2018 and 2019 far less restaurants pass an inspection in favour of passing with conditions.
Furthermore we want to see how many inspections results correspond to which Risk level.

In [ ]:
data = inspections.groupby([inspections["Inspection Date"].dt.year, inspections["Risk"]])["Inspection ID"].count().to_frame().reset_index()
data = data.rename(columns={"Inspection ID": "Number of inspections"})
sns.barplot(data["Inspection Date"], data["Number of inspections"], data["Risk"]).set_title("Number of inspections per year by risk level")

We can see that most of inspections end up with giving restaurants a high risk badge, there are very few inspections with low risk, this is even more interesting as we identified earlier that most inspections are the ones that were passed.

## Research questions

Below we present data analysis for each of the research questions we defined. These analysis leads to initial answers to these questions, we also describe one question we decided to drop (see Question 5). At the end of data analysis for each research question, we briefly describe what will/can be done with regards to the posed question.

### Question 1
**What are the most common reasons for a restaurant not passing an inspection?**

In our dataset violations detected during the inspections are noted in the Violations column. Each violation contains a code, general violation description (name/category of the violation) and comments added by the inspector. Since the violation codes uniquely identify a particular violation, we extract them and use them for our analysis.

However, as indicated in the [linked document](http://dev.cityofchicago.org/open%20data/data%20portal/2018/06/29/food-violations-changes.html) from the dataset page, the violation codes and descriptions has changed as of 1/07/2018. This means, that violation codes before and after this date don't correspond to the same violation reason. We first tried to map the changed values to the old ones, but it did not give results we expected (there are many new violations added and also the meaning of many violation changed). Therefore we need to split our analysis into two parts, so that violation codes in each part are consistent - analyze the inspections before 1/07/2018 and after this day.

What we also wanted to do in this part was to categorize these violation reasons into bigger, more general groups. Categorizing violations into smaller number of groups would help us understand the highlevel reasons for failing the inspection better and would also help in creating visualizations easier to understand. That is why we manually grouped all violation codes into **5 categories** (using the violation descriptions for understanding what each violation code means):
1. **Food violations:** in this category we put all of the violations which are related to the food and ingredients: the way they are obtained, stored, transported, prepared, labeled etc.
2. **Facility conditions related violations:** presence of appropriatelly set up equipment and utilities (freezer, owens, work surface etc.), as well as requirements for the building and infrastructure (lighting, ventilation, temperature, pipes, walls etc.).
3. **Sanitary related violations:** violations that are related to keeping all the equipment, rooms and surfaces clean .
4. **Staff related violations:** violations related to the employees (necesarry trainings, the way they work with food, manager's work etc.).
5. **Other:** the violations that did not match the previous groups (eg. summary report of the inspection visible to the public, no smoking regulations etc.).

#### Analysis on original violation codes

To perform analysis on violations we first need to extract their codes to easily identify violations for each inspection:

In [ ]:
inspections.fillna(value={'Violations': ''}, inplace=True)

In [ ]:
# Extract violation codes from textual list of violations
def extract_violation_codes(violation):
    violations_list = list(map(lambda v: v.strip(), violation.split('|')))
    violation_dots = [violation.find('.') for violation in violations_list]
    # using set to get rid of multiple same category violations per inspection (to not disturb the distributions)
    violation_codes = [int(v[:idx]) for v, idx in zip(violations_list, violation_dots) if idx != -1]
    return violation_codes

In [ ]:
# Add additional column with extracted codes
inspections['Violation Codes'] = inspections['Violations'].apply(extract_violation_codes)

In [ ]:
# Merges all violation codes from the column into one flat array
def merge_violation_codes(violation_series):
  return [code for inspection_violation_codes in violation_series.values for code in inspection_violation_codes]

# Creates the histogram for violation codes
def violation_counts(violations, max_violation_code):
  counts, code_bins = np.histogram(violations, bins=np.arange(1, max_violation_code + 2))
  return counts, code_bins

# Creates violation codes distribution from the dataframe
def violations_distribution(df, violation_column='Violation Codes', max_violation_code=70):
  # https://stackoverflow.com/a/38258158
  all_codes = merge_violation_codes(df[violation_column])
  counts, code_bins = violation_counts(all_codes, max_violation_code)
  return code_bins[:-1], counts

In [ ]:
# Split the dataset regarding the change of violation definitions
violation_change_date = pd.Timestamp(year=2018, month=7, day=1)
inspections['Inspection Date'] = pd.to_datetime(inspections['Inspection Date'])
inspections_before_change = inspections[inspections['Inspection Date'] < violation_change_date]
inspections_after_change = inspections[inspections['Inspection Date'] >= violation_change_date]

In [ ]:
def plot_violations_stacked_bars(data, title, violation_column='Violation Codes', max_violation_code=70, xticks=None):
  plt.figure(figsize=(12, 7))
  plt.title(title)

  results = data['Results'].unique()
  bars = []
  previous_counts = np.zeros((max_violation_code,))
  # create stacked bar chart
  for result in results:
    bins, counts = violations_distribution(data[data['Results'] == result], violation_column, max_violation_code)
    bar = plt.bar(bins, counts, bottom=previous_counts)
    bars.append(bar[0])
    # accumulate counts for positioning next stack
    previous_counts += counts
  
  if xticks is not None:
    plt.xticks(np.arange(1, max_violation_code + 1), xticks, rotation=40)
  else:
    plt.xlabel('Violation code')

  plt.ylabel('# of inspections with violations')
  plt.ylim((0, int(np.max(previous_counts)) + 2000))

  plt.legend(tuple(bars), tuple(results))
  plt.grid(True, axis='y')

  plt.show()

In [ ]:
plot_violations_stacked_bars(inspections_before_change, 'Distribution of most common violations before 1/07/2018')

In [ ]:
plot_violations_stacked_bars(inspections_after_change, 'Distribution of most common violations after 1/07/2018')

In [ ]:
# Focus only on failed inspections
failed_inspections_before = inspections_before_change[inspections_before_change['Results'] == 'Fail']
failed_inspections_after = inspections_after_change[inspections_after_change['Results'] == 'Fail']

In [ ]:
plot_violations_stacked_bars(failed_inspections_before, 'Failed inspection violations distribution before 1/07/2018')

In [ ]:
plot_violations_stacked_bars(failed_inspections_after, 'Failed inspection violations distribution after 1/07/2018')

The plots above show what are the most common violations for inspections both in general and only when focusing on failed inspections. In the first plot we can also see that violation with code 18 is almost always associated with a failed inspection. In the same plot it is also appartent, that violations with codes 32-35 and 38 are very often, but also most of such inspections end with a pass. We can also note that after the change of violations there are indeed not that many inspections with some violations and still getting a clear pass.

#### Analysis on generalized inspection codes

Now we convert the original violation codes into defined 5 categories:

In [ ]:
food_codes_before = [1, 3, 15, 16, 17, 30]
facility_codes_before = [2, 7, 9, 11, 13, 18, 22, 23, 24, 25, 26, 29, 32, 35, 36, 37, 38, 40, 42, 43]
sanitary_codes_before = [4, 8, 10, 12, 19, 20, 27, 31, 33, 34, 39, 41]
staff_codes_before = [5, 6, 21, 44, 45]
unknown_codes_before = [14, 28, 70]

codes_before = [food_codes_before, facility_codes_before, sanitary_codes_before, staff_codes_before, unknown_codes_before]

In [ ]:
food_codes_after = [11, 12, 13, 14, 15, 17, 23, 26, 27, 28, 30, 31, 37, 39, 42]
facility_codes_after = [10, 18, 19, 20, 21, 22, 33, 35, 36, 38, 41, 43, 44, 48, 50, 51, 53, 55, 56, 59, 60, 62]
sanitary_codes_after = [2, 8, 16, 40, 45, 46, 47, 49, 52, 54]
staff_codes_after = [1, 3, 7, 9, 25, 57, 58]
unknown_codes_after = [4, 5, 6, 24, 29, 32, 61, 63]

codes_after = [food_codes_after, facility_codes_after, sanitary_codes_after, staff_codes_after, unknown_codes_after]

In [ ]:
# Based on the above array creates mappings between original codes and generalized categories
def create_mapping(codes):
  mapping = {}
  for new_code, category_codes in enumerate(codes):
      for old_code in category_codes:
          mapping[old_code] = new_code + 1  # new codes starting from 1
  return mapping

# Function to transform array of original codes into our categories
def encode_violations(violations, mapping):
  encoded = []
  for violation in violations:
    encoded.append(mapping[violation])
  return encoded

In [ ]:
# Create mappings for conversion
before_mapping = create_mapping(codes_before)
after_mapping = create_mapping(codes_after)

In [ ]:
mapped_inspections_before_change = inspections_before_change['Violation Codes'].apply(encode_violations, mapping=before_mapping)
mapped_inspections_after_change = inspections_after_change['Violation Codes'].apply(encode_violations, mapping=after_mapping)

In [ ]:
mapped_inspections = mapped_inspections_before_change.append(mapped_inspections_after_change)
# Add additional column with violation codes by our general categories
inspections = inspections.merge(mapped_inspections, left_index=True, right_index=True, suffixes=('', ' Generalized'))
inspections.head()

#### Analysis on critical violations

Not all the violations have the same weight, right? For instance, it is not the same to have dirty toilet sink and serve expired food. Therefore, we want to make a distinction between violations, not only based on the category, but based on how serous the violation was.

In order to differentiate restaurants and neighborhoods based on the severity of the violations they make, we need to know whether in some inspection there were some critical violations broken. For this, our analysis again differs before and after 1.7.2018. 

In the official description of the dataset, which applies to violations before 1st of July 2018, it is stated that violations with code 1-14 are classified as **critical violations**, 15-29 are **serious violations**, and the rest are **minor violations**. 

[After the change of the violations in 2018](https://www.chicago.gov/content/dam/city/depts/cdph/food_env/general/Food_Protection/2019_ChicagoFoodCodeMajorChanges.pdf), different renaming of critical, serious and minor violation was applied and they are called **priority**, **priority foundation** and **core violation** respectively. Moreover, there was no more clear separation which violation code belongs to which group. The way to deduct the severity of the violation in each inspection is to look at the **comments made by inspectors.** If there was a critical violation broken, it would be marked as priority violation.

We will use this information to check if there was any critical or serious violation for each inspection. We will create 2 new columns which will contain boolean values: Critical Violation Noticed and Serious Violation Noticed. 

In [ ]:
# Function that applies for inspections before change (1.7.2018) and checks if there is any critical violation among given violation codes 
def is_critical_violation_before_change(violation_codes):
    critical_violations = np.arange(1,15).tolist()
    return (list(set(violation_codes) & set(critical_violations)) != [])

# Function that applies for inspections after change (1.7.2018) and checks if there is any critical violation among given violation codes 
def is_critical_violation_after_change(comment):
    return ('priority violation' in comment.lower())

# Function that checks if during the inspection it was observed at least one critical violation
def check_critical_violation(row):
    change_date = datetime.datetime.strptime('2018-07-01', '%Y-%m-%d')
    if((row['Inspection Date'] < change_date) and (is_critical_violation_before_change(row['Violation Codes']))):
        return True
    if((row['Inspection Date'] >= change_date) and (is_critical_violation_after_change(row['Violations']))):
        return True
    return False

In [ ]:
# Create new column which shows if during the inspection it was observed at least one critical violation
inspections['Critical Violation Noticed'] = inspections.apply(lambda x : check_critical_violation(x), axis = 1)

Similarily, we will check serious violations. They are slighlty less dangerous than critical ones, but still considered harmful for human's health.

In [ ]:
# Function that applies for inspections before change (1.7.2018) and checks if there is any serious violation among given violation codes 
def is_serious_violation_before_change(violation_codes):
    serious_violations = np.arange(15,30).tolist()
    return (list(set(violation_codes) & set(serious_violations)) != [])

# Function that applies for inspections after change (1.7.2018) and checks if there is any critical violation among given violation codes 
def is_serious_violation_after_change(comment):
    return ('priority foundation' in comment.lower())

# Function that checks if during the inspection it was observed at least one serious violation
def check_serious_violation(row):
    change_date = datetime.datetime.strptime('2018-07-01', '%Y-%m-%d')
    if((row['Inspection Date'] < change_date) and (is_serious_violation_before_change(row['Violation Codes']))):
        return True
    if((row['Inspection Date'] >= change_date) and (is_serious_violation_after_change(row['Violations']))):
        return True
    return False

In [ ]:
# Create new column which shows if during the inspection it was observed at least one serious violation
inspections['Serious Violation Noticed'] = inspections.apply(lambda x: check_serious_violation(x), axis=1)

In [ ]:
inspections.head()

In [ ]:
critical_violations_percentage = int(100*inspections[inspections['Critical Violation Noticed']].shape[0]/ inspections.shape[0])
print('{0}% of inspections had at least one critical violation.'.format(critical_violations_percentage))

In [ ]:
serious_violations_percentage = int(100*inspections[inspections['Serious Violation Noticed']].shape[0]/ inspections.shape[0])
print('{0}% of inspections had at least one serious violation.'.format(serious_violations_percentage))

In [ ]:
# Plot proportion of critical and serious violations in inspections
fig = plt.figure()
plt.suptitle('Percentage of inspections having critical or serious violations');

startingRadius = 1
# First donut plot - Plotting proportion of critical violations
ax1 = fig.add_subplot(221)
critical_violation_donut = [100-critical_violations_percentage, critical_violations_percentage]
ax1.text(0.13, startingRadius + 0.17, "Critical violations: "+str(critical_violations_percentage)+"%", 
         horizontalalignment='center', verticalalignment='center')
ax1.pie(critical_violation_donut, radius=startingRadius, startangle=90, colors=['#fce0de', '#ed5b51'],
            wedgeprops={"edgecolor": "white", 'linewidth': 1})
circle = plt.Circle(xy=(0, 0), radius=0.45, facecolor='white')
plt.gca().add_artist(circle)

# Second donut plot - Plotting proportion of serious violations
ax2 = fig.add_subplot(222)
ax2.text(0.13, startingRadius + 0.17, "Serious violations: "+str(serious_violations_percentage)+"%", 
         horizontalalignment='center', verticalalignment='center')
serious_violation_donut = [100-serious_violations_percentage, serious_violations_percentage]
ax2.pie(serious_violation_donut, radius=startingRadius, startangle=90, colors=['#fce0de', '#f0776e'],
            wedgeprops={"edgecolor": "white", 'linewidth': 1})
circle = plt.Circle(xy=(0, 0), radius=0.45, facecolor='white')
plt.gca().add_artist(circle)

plt.show()

#### Generalized violations

Apart from grouping violations in 5 categories, we also named all of the validations, both before and after change. This is going to help us in better visualization of the violations.

In [ ]:
violations_before_change = pd.read_excel(os.path.join(data_folder,'violation_names_before_2018.xlsx'))
violations_before_change.head()

Since we want to focus on the most common reasons for failing an inspection, we want to count how many times each violation occured in inspections that ended in failure.

In [ ]:
# Create list of all violation codes found during failed inspections
all_violations_before_change_list = failed_inspections_before['Violation Codes'].values.tolist()
all_violations_before_change_list = [item for sublist in all_violations_before_change_list for item in sublist]

# Count occurencies of each validation
from collections import Counter
counted_violations_before_change = Counter(all_violations_before_change_list)
counted_violations_before_change = pd.DataFrame.from_dict(counted_violations_before_change, orient='index').reset_index()

violations_before_change = violations_before_change.merge(counted_violations_before_change, left_on = 'Violation code', right_on ='index').sort_values(by=0,ascending = True).rename(columns={0:'# violated'}).drop(columns='index')
violations_before_change.head()

Apart from getting the number of violation, we will enrich this data frame with three additional columns: Violation Category, Critical Violation and Serious Violation. Violation Category will be one of the previously mentioned 5 categories that we created. Critical Violation and Serious Violation are boolean valued columns saying whether violation is critical or serious, respectively. As we explained, validation codes 1-14 are critical violations, and 15-29 are serious violations. Note that we won't be able to create those two columns for the violations after change.

In [ ]:
# Function that returns violation category for violations before 1.7.2018.
def get_violation_category_before_change(violationCode):
    # Definiton of code mappings
    food_codes_before = [1, 3, 15, 16, 17, 30]
    facility_codes_before = [2, 7, 9, 11, 13, 18, 22, 23, 24, 25, 26, 29, 32, 35, 36, 37, 38, 40, 42, 43]
    sanitary_codes_before = [4, 8, 10, 12, 19, 20, 27, 31, 33, 34, 39, 41]
    staff_codes_before = [5, 6, 21, 44, 45]
    unknown_codes_before = [14, 28, 70]
    
    if (violationCode in food_codes_before):
        return 'Food related violations'
    if (violationCode in facility_codes_before):
        return 'Facility related violations'
    if (violationCode in sanitary_codes_before):
        return 'Sanitary related violations'
    if (violationCode in staff_codes_before):
        return 'Staff related violations'
    if (violationCode in unknown_codes_before):
        return 'Other'
    return None

In [ ]:
# Create columns - Violation Category, Critical Violation & Serious Violation
violations_before_change['Violation Category'] = violations_before_change['Violation code'].apply(lambda x: get_violation_category_before_change(x))
violations_before_change['Critical Violation'] = violations_before_change['Violation code'].apply(lambda x: is_critical_violation_before_change([x]))
violations_before_change['Serious Violation'] = violations_before_change['Violation code'].apply(lambda x: is_serious_violation_before_change([x]))
violations_before_change.head()

Now that we prepared the data for the analysis, we first want to see which violations are the ones that are present in the biggest number of failed inspections. We will plot all the violations names and the corresponding number of violations made, as well as the category to which that violation belongs to.

In [ ]:
import plotly.graph_objects as go

categories = ['Food related violations', 'Facility related violations','Sanitary related violations','Staff related violations','Other']

fig = go.Figure()

for category in categories:
    fig.add_trace(go.Bar(x=violations_before_change[violations_before_change['Violation Category']==category]['# violated'],
                y=violations_before_change[violations_before_change['Violation Category']==category]['Violation name'],
                name=category,
                orientation='h',
                hovertemplate='%{y}<br>Number of violations: %{x}',
                # choose color palette
                #marker_color='rgb(55, 83, 109)'
                ))
fig.update_layout(
    title={
        'text': "Number of violations for failed inspection result",
        #'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Violation name',
        titlefont_size=16,
        tickfont_size=10,
        dtick= 1,
        categoryorder="total ascending"
    ),
    xaxis_title="Number of violations",
    bargap=0.4,
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    #height = 1200,
    #width = 1200
)

In [ ]:
## TODO: SAVE PLOT
#pyo.plot(fig, filename='figure.html',validate=False)
#from IPython.display import IFrame
#IFrame(src='./figure.html', width=1000, height=600)

We have a winner! Among the failed inspections, **dirty floor was mentioned the most** as one of the violations. Two out of top three violations are related to cleaningness in the restaurant. Interestingly, many restaurants have problems with walls and ceiling constructions. Due to the fact that we created the categories by ourselves and that we are assigning each violation to strictly one category, there could be that some part of ceiling and wall problems is also due to dirtiness apart from construction. 

It seems that "most popular violations" are sanitary and facility related. What suprises is that, even though we are analysing restaurants, there is only one food related violation among the top 10 and it is on 10th place.

The violations that restaurants made the least were re-serving the food and not cleaning the dishes properly. What is for sure very important to all of us who visit or want to visit restaurants in Chicago is that employees do not work if they are sick or injured - or at least inspection rarely finds out that.

Looking at the top violations in failing, the inadequate floor or ceiling maitenance does not seem so severe. Let's now focus only on critical violations and see what is the most frequent critical violation.

In [ ]:
# Visualize critical violations
fig = px.bar(violations_before_change[violations_before_change['Critical Violation']], 
             x="# violated", y="Violation name", labels={'# violated':'Number of violations'},
            orientation='h',color='Violation Category')
fig.update_layout(
    title="Number of critical violations for failed inspection result",
    xaxis_title="Number of violations",
    yaxis_title="Critical violation name",
    yaxis_dtick= 1,
    bargap=0.2,
    yaxis_categoryorder = "total ascending"
)

When looking at critical violations only, the top violation is related to **proper food storage temperature**. According to the full violation description, it means that potentially hazardous food meets temperature requirements during storage, preparation and service. On second and third place we have facility related violations, and then come the ones related to adequate cleaning of all necessary areas and equipement.

In [ ]:
# Visualize serious violations
fig = px.bar(violations_before_change[violations_before_change['Serious Violation']], 
             x="# violated", y="Violation name", labels={'# violated':'Number of violations'},
            orientation='h',color='Violation Category')
fig.update_layout(
    title="Number of serious violations for failed inspection result",
    xaxis_title="Number of violations",
    yaxis_title="Critical violation name",
    yaxis_dtick= 1,
    bargap=0.2,
    yaxis_categoryorder = "total ascending"
)

Among the serious violations, the big problem for restaurants is that they do not have adequate protection from entrance of rodents and insects in the building. Also, many restaurants do not manage to correct serious violations they made, which also brings up the possibility to think about the performance of restaurants on re-inspections, which we will discuss later.

Now, we will do the same analysis for the violations after the change on 1.7.2018.

In [ ]:
violations_after_change = pd.read_excel(os.path.join(data_folder,'violation_names_after_2018.xlsx'))
violations_after_change.head()

In [ ]:
# Create list of all violation codes found during failed inspections
all_violations_after_change_list = failed_inspections_after['Violation Codes'].values.tolist()
all_violations_after_change_list = [item for sublist in all_violations_after_change_list for item in sublist]

# Count occurencies of each validation
counted_violations_after_change = Counter(all_violations_after_change_list)
counted_violations_after_change = pd.DataFrame.from_dict(counted_violations_after_change, orient='index').reset_index()

violations_after_change = violations_after_change.merge(counted_violations_after_change, left_on = 'Violation code', right_on ='index').sort_values(by=0,ascending = False).rename(columns={0:'# violated'}).drop(columns='index')
violations_after_change.head()

In [ ]:
# Function that returns violation category for violations before 1.7.2018.
def get_violation_category_after_change(violationCode):
    # Definiton of code mappings
    food_codes_after = [11, 12, 13, 14, 15, 17, 23, 26, 27, 28, 30, 31, 37, 39, 42]
    facility_codes_after = [10, 18, 19, 20, 21, 22, 33, 35, 36, 38, 41, 43, 44, 48, 50, 51, 53, 55, 56, 59, 60, 62]
    sanitary_codes_after = [2, 8, 16, 40, 45, 46, 47, 49, 52, 54]
    staff_codes_after = [1, 3, 7, 9, 25, 57, 58]
    unknown_codes_after = [4, 5, 6, 24, 29, 32, 61, 63]
    
    if (violationCode in food_codes_after):
        return 'Food related violations'
    if (violationCode in facility_codes_after):
        return 'Facility related violations'
    if (violationCode in sanitary_codes_after):
        return 'Sanitary related violations'
    if (violationCode in staff_codes_after):
        return 'Staff related violations'
    if (violationCode in unknown_codes_after):
        return 'Other'
    return None

In [ ]:
# Create columns - Violation Category, Critical Violation & Serious Violation
violations_after_change['Violation Category'] = violations_after_change['Violation code'].apply(lambda x: get_violation_category_after_change(x))
violations_after_change.head()

In [ ]:
# Visualising only 40 most frequent violations
top_violations_after_change = violations_after_change.iloc[0:40,:].copy()

categories = ['Food related violations', 'Facility related violations','Sanitary related violations','Staff related violations','Other']

fig = go.Figure()

for category in categories:
    fig.add_trace(go.Bar(x=top_violations_after_change[top_violations_after_change['Violation Category']==category]['# violated'],
                y=top_violations_after_change[top_violations_after_change['Violation Category']==category]['Violation name'],
                name=category,
                orientation='h',
                hovertemplate='%{y}<br>Number of violations: %{x}',
                # choose color palette
                #marker_color='rgb(55, 83, 109)'
                ))
fig.update_layout(
    title={
        'text': "Number of violations for failed inspection result",
        #'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Violation name',
        titlefont_size=16,
        tickfont_size=10,
        dtick= 1,
        categoryorder="total ascending"
    ),
    xaxis_title="Number of violations",
    bargap=0.4,
    legend=dict(
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    #height = 1400,
    #width = 1400
)

We already mentioned the big differences and stricter criteria after the change date. That can also be noticed in the violations that are made the most. Now for the first time we can see a big impact of **violations related to employees training**. Apparently, many restaurants did not have time do adjust properly to the new regulations and failed in the past year and a half due to inadequate management or lack of allergen training for the staff. Also, the City of Chicago requires defined procedures for many situations that can happen in a restaurant, and one that is the most problematic for restaurants to comply with is **reacting in case of customer sickness.** So be careful and remind yourself of your first aid knowledge, because the restaurant employees apparently cannot help you properly. Of course, when looking at these reasons, we have to take into account that these changed violation list is valid for less than 18 months, so there is smaller amount of inspections which can be analyzed. 

## TODO: Check if we are keeping this part

Now we can present similar plots as before, but for generalized categories. Also, now we can plot the distributions of violations in these categories for our entire dataset on the same plot:

In [ ]:
labels = ['Food', 'Facility conditions', 'Sanitary conditions', 'Staff', 'Other']
plot_violations_stacked_bars(inspections, '', violation_column='Violation Codes Generalized', max_violation_code=5, xticks=labels)

We can see that most inspections with violations are related to the violations we clasiffied as facility conditions. Clearly the reason is mostly the fact, that this category is mapped to from many of the original violation codes. Original violation codes assigned to new category "Facility conditions" form the biggest groups within the full sets of original violation codes.

It will be also interesting to analyze violations in inspection results by neighbourhood to potentially discover patterns (e.g. areas larger than one neighbourhood that have inspections with particular violation as the most common one). Below are the functions for perfoming such analysis:

In [ ]:
# Get violation codes list for each neighbourhood (zip code)
def violation_codes_by(data, by_column, violation_column):
  return data.groupby(by=by_column)[violation_column].apply(merge_violation_codes)

# Find top most common violations in given violations list
def most_common_violations(violations, max_violation_code, top=1):
  counts, _ = violation_counts(violations, max_violation_code)
  return np.argsort(counts)[::-1][:top] + 1  # add one to convert to violation code from index

# Find top most common violations in each neighbourhood (zip code)
def most_common_violations_by(data, by_column, violation_column, max_violation_code, top=1):
  return violation_codes_by(data, by_column, violation_column).apply(most_common_violations, max_violation_code=max_violation_code, top=top)

#### Milestone 3 goals

For the next milestone we are going to analyze spatial patterns within Chicago for the most common violations per neighborhood. Also, violations discovered during inspections can serve as another measure for ranking places where to eat in Chicago, i.e. we can use percentage of passed inspections or (more detailed) percentage of passed inspections with no food related violations for gauging the quality and safety of restaurants in particular neighbourhood. Establishing such additional measures may help us in responding to the main project question based on not that commonly analyzed aspects and propose safe to eat neighbourhoods based on such measure.

In [ ]:
by = 'Community'
most_common_violations_by_community_before_change = most_common_violations_by(inspections_before_change, by, 'Violation Codes', 70, top=1)
most_common_violations_by_community_after_change = most_common_violations_by(inspections_after_change, by, 'Violation Codes', 63, top=1)
most_common_generalized_violations_by_community = most_common_violations_by(inspections, by, 'Violation Codes Generalized', 5, top=1)

In [ ]:
most_common = pd.DataFrame(most_common_generalized_violations_by_community) \
    .rename(columns={'Violation Codes Generalized': 'general_most_common'}) \
    .merge(most_common_violations_by_community_before_change, left_index=True, right_index=True) \
    .rename(columns={'Violation Codes': 'before_most_common'}) \
    .merge(most_common_violations_by_community_after_change, left_index=True, right_index=True) \
    .rename(columns={'Violation Codes': 'after_most_common'}) \
    .reset_index()

In [ ]:
# Transform most common general violation to value from list
most_common = most_common.explode('general_most_common')
most_common['community_name'] = most_common['Community'] # for plotting purposes

In [ ]:
# Transform violation codes into names
def transform_violation_codes_into_names(violation_codes, names_mapping_df):
    names_array = [names_mapping_df[names_mapping_df['Violation code'] == code]['Violation name'].values for code in violation_codes]
    if len(names_array) > 0:
        return ', '.join([names[0] for names in names_array])
    else:
        return 'None'

most_common['before_most_common_names'] = most_common['before_most_common'].apply(transform_violation_codes_into_names, names_mapping_df=violations_before_change)
most_common['after_most_common_names'] = most_common['after_most_common'].apply(transform_violation_codes_into_names, names_mapping_df=violations_after_change)

In [ ]:
most_common_before = most_common.copy().drop(columns=['general_most_common', 'after_most_common', 'after_most_common_names']).explode('before_most_common')
most_common_after = most_common.copy().drop(columns=['general_most_common', 'before_most_common', 'before_most_common_names']).explode('after_most_common')
most_common.drop(columns=['before_most_common', 'after_most_common'], inplace=True)

In [ ]:
# reordering columns for plotting purposes
most_common = most_common[['Community', 'general_most_common', 'community_name', 'before_most_common_names', 'after_most_common_names']]

In [ ]:
from plotting import plot_map

%load_ext autoreload
%autoreload 2

In [ ]:
mapbox_access_token = 'pk.eyJ1IjoiYnpkZWNvIiwiYSI6ImNrNDVmNTU5ZTA1dnIzZXJ2Zm9vbmZsd2EifQ.KvNq66ornrV9ACa98F0s9w'

In [ ]:
template = '<b>{}</b><br><b>Before 1/07/2018:</b> {}<br><b>After 1/07/2018:</b> {}'

In [ ]:
# TODO: set discrete colorscale
plot_map('Most common violations for Chicago communities', 
         geodata_path='data/chicago-community-areas.json', 
         dataframe=most_common, 
         key_property='community', key_column='Community', value_column='general_most_common',
         template=template,
         mapbox_access_token=mapbox_access_token,
         options={})

#### Most common violations before change of violations

In [ ]:
# https://community.plot.ly/t/colors-for-discrete-ranges-in-heatmaps/7780
# Creates a colorscale with discrete steps for plotting given start and end rgb tuples and number of discrete values
def make_discrete_colorscale(start, end, n_categories):
    rs, gs, bs = start
    re, ge, be = end
    r, g, b = (re - rs)/(n_categories - 1), (ge - gs)/(n_categories - 1), (be - bs)/(n_categories - 1)
    colors = []
    for cat in range(n_categories):
        for i in range(2):
            colors.append([(cat + i) / n_categories, f'rgb({rs + cat*r}, {gs + cat*g}, {bs + cat*b})'])
    return colors

In [ ]:
# Map violation codes to ordered values starting from 0
def map_violations_to_discrete_ordered_values(dataframe, mapped_column):
    original_values = list(dataframe[mapped_column].unique())
    mapped_to = list(range(len(original_values)))
    mapping = dict(zip(original_values, mapped_to))  # https://stackoverflow.com/a/209854
    dataframe[mapped_column] = dataframe[mapped_column].map(mapping)

In [ ]:
map_violations_to_discrete_ordered_values(most_common_before, 'before_most_common')

In [ ]:
# Plot the map
n_categories = len(most_common_before['before_most_common'])
template = '<b>{}</b><br>{}'
plot_map('Most common violations for Chicago communities before 1/07/2018', 
         geodata_path='data/chicago-community-areas.json', 
         dataframe=most_common_before, 
         key_property='community', key_column='Community', value_column='before_most_common',
         template=template,
         mapbox_access_token=mapbox_access_token,
         options={
             'colorscale': make_discrete_colorscale((200, 200, 255), (0, 0, 255), n_categories),
             'showscale': False
         })

#### Most common violations after change of violations

In [ ]:
map_violations_to_discrete_ordered_values(most_common_after, 'after_most_common')

In [ ]:
# Plot the map
n_categories = len(most_common_after['after_most_common'])
template = '<b>{}</b><br>{}'
plot_map('Most common violations for Chicago communities after 1/07/2018', 
         geodata_path='data/chicago-community-areas.json', 
         dataframe=most_common_after, 
         key_property='community', key_column='Community', value_column='after_most_common',
         template=template,
         mapbox_access_token=mapbox_access_token,
         options={
             'colorscale': make_discrete_colorscale((200, 200, 255), (0, 0, 255), n_categories),
             'showscale': False
         })

#### Percentage of inspections with critical violations per community

In [ ]:
def percentages_of_violation_types(by='Community', violation_type='Critical Violation Noticed'):
    group_size = inspections.groupby(by)[violation_type].count()
    counts_per_group = inspections[inspections[violation_type]].groupby(by)[violation_type].count()
    return 100 * counts_per_group / group_size

In [ ]:
by = 'Community'
critical_percentages = percentages_of_violation_types(by, violation_type='Critical Violation Noticed')

In [ ]:
serious_percentages = percentages_of_violation_types(by, violation_type='Serious Violation Noticed')

In [ ]:
critical_percentages_df = pd.DataFrame(critical_percentages) \
    .rename(columns={'Critical Violation Noticed': 'percentage'}) \
    .reset_index() \
    .round(decimals=1)
critical_percentages_df['community_name'] = critical_percentages_df['Community']
critical_percentages_df['percentage_value'] = critical_percentages_df['percentage']

In [ ]:
serious_percentages_df = pd.DataFrame(serious_percentages) \
    .rename(columns={'Serious Violation Noticed': 'percentage'}) \
    .reset_index() \
    .round(decimals=1)

In [ ]:
template = '<b>{}</b>: {}%'
plot_map('Percentage of inspections with critical violations per community', 
         geodata_path='data/chicago-community-areas.json', 
         dataframe=critical_percentages_df, 
         key_property='community', key_column='Community', value_column='percentage',
         template=template,
         mapbox_access_token=mapbox_access_token,
         options={
             'colorscale': [
                 [0.0, 'rgb(255, 255, 0)'],
                 [1.0, 'rgb(255, 0, 0)']
             ],
             'colorbar': {
                 'title': 'Percentage',
                 'tick0': 0,
                 'dtick': 4,
                 'ticksuffix': '%'
             }
         })

### Question 2
**Are there "safe to eat" areas or "dangerous to eat" areas in Chicago?**

As a first criteria for measuring whether neighborhood is "safe to eat" or "dangerous to eat", we will take into account the aggregate results of inspections for the restaurants in that area.

In [ ]:
inspections['Results'].unique()

In [ ]:
# Get contigency table
inspections_scores_by_zip = inspections.groupby(['Zip','Results']).size().unstack('Results', fill_value=0)
inspections_scores_by_zip.head()

We decided to create a formula which will give the overall score for each neighborhood, taking into account the number of inspections per each category, as well as penalizing the negative results and awarding the positive ones. 
If the restaurant was suspended, which means Out of Business or Business Not Located (according to the docs), that gives -2 point. If the restaurant failed, it will be -1 points. If it passes with condition, the score is 0.5 points. If the restaurant passed clearly, it gives 1 point. We then divide that by the total amount of restaurants in the neighborhood.

In [ ]:
# Function that gives the safety score for the neighborhood
def get_safety_score(df):
    score = (-2)*df['Business Not Located']+(-2)*df['Out of Business']+(-1)*df['Fail']+0.5*df['Pass w/ Conditions']+1*df['Pass'] 
    number_of_inspections = df.sum(axis=1)
    return score/number_of_inspections

In [ ]:
# Add safety score to DF
inspections_scores_by_zip['Safety score'] = pd.Series(get_safety_score(inspections_scores_by_zip))
inspections_scores_by_zip.sort_values('Safety score', ascending=False).head()

Now, we want to visualise the results in order to see if there is any pattern.

In [ ]:
# We need Zip to be a column in order to visualize, and Zip must be string
inspections_scores_by_zip.reset_index(level=0,inplace=True)
inspections_scores_by_zip['Zip'] = inspections_scores_by_zip['Zip'].apply(lambda x: str(x))

In [ ]:
# Visualise neighorhoods with respect to the Safety Score they got
state_geo_path = os.path.join(data_folder, "chicago-zip.json")
geo_json_data = json.load(open(state_geo_path))

map_safe_danger = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
map_safe_danger.choropleth(geo_data=geo_json_data, data=inspections_scores_by_zip,
             columns=['Zip', 'Safety score'],
             key_on='feature.properties.zip',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Safe to eat percentage (%)')
map_safe_danger
#map_safe_danger.save('safe-vs-dangerous-map.html')

According to the safety scores we calculated, it seems that the safest places to eat are the **the two Airports and the City Center.** That's right, you may need to pay extra, since those are central tourist points, but you can be quite sure that the restaurant you are eating is fulfulling the regulations. As you are going further from this places, the risk increases. 


#### Milestone 3 goals

We plan to explore further the "safe vs. dangerous" areas by choosing several criteria. We plan to check how Risk feature impacts that separation, as well as try to find other metrics which could be relevant for this analysis.

### Question 3
**Checking restaurants history of inspections, how are restaurants or whole areas changing in their inspection scores? Can one see any patterns in improvements with respect to inspection results? Are there areas/restaurant chains/restaurant types that follow some trends?**

Let's start by a look at the possible inspection results :

In [ ]:
ax = sns.countplot(inspections.Results)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
plt.tight_layout()

We will ignore out of service restaurants, and therefore only consider the first three categories: Pass, Pass w/ conditions and Fail.

As a first indicator of restaurant quality, we will use the number of safety violations per inspection.

In [ ]:
inspections["nb_violations"] = inspections.apply(lambda row: len(row['Violation Codes']), axis=1)
inspections.sample(5)

Let's compare the number of violations for inspections with results "Pass", "Pass w/ conditions" and "Fail" :

In [ ]:
inspections[inspections.Results == "Pass"].nb_violations.describe()

In [ ]:
inspections[inspections.Results == "Fail"].nb_violations.describe()

As expected, the average number of violations is higher for restaurants who failed their inspection. However, an important detail shouldn't be overlooked: quartiles show that some failing restaurants have less violations than passing ones. We will therefore consider two parameters to evaluate how much restaurants improve over time: the difference between the intial average number of violations and the same average number at the last inspection, and their inspection result (Pass, Pass w/ conditions and Fail). 

In [ ]:
def nvdiff(group):
    tmp = group.sort_values(by="Inspection Date")
    return (tmp.iloc[-1].nb_violations-tmp.iloc[0].nb_violations)
violations_evo = inspections.groupby("License #").apply(nvdiff)
violations_evo = pd.DataFrame({'License #':violations_evo.index, 'Violations evolution':violations_evo.values})

In [ ]:
violations_evo.sample(5)

We can now compute the average evolution score per zip code (which is the average of the difference between the number of violations at first and last inspections for all restaurants of a given area).

In [ ]:
violations_evo_zip = violations_evo.merge(inspections[['License #','Zip']],on='License #', how='left')\
                         .groupby('Zip')['Violations evolution'].mean()
violations_evo_zip = pd.DataFrame({'Zip':violations_evo_zip.index,'evolution':violations_evo_zip.values})
violations_evo_zip.sample(5)

In [ ]:
state_geo_path = os.path.join(data_folder, "chicago-zip.json")
geo_json_data = json.load(open(state_geo_path))

map_evo = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
map_evo.choropleth(geo_data=geo_json_data, data=violations_evo_zip,
             columns=['Zip', 'evolution'],
             key_on='feature.properties.zip',
             bins=[-4.0,-3.0,-2.0,0.0,1.0,3.0],
             fill_color='BuGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Violations number evolution')
map_evo

We can see that in most areas, restaurants manage to decrease their number of violations over time. But in a few areas, restaurants have increased their number of violations.

We can also visualize the number of restaurants who have improved, got worse or stagnated over time.

In [ ]:
bins = [-np.inf,-0.1,0.1,np.inf]
cats = ["Improved", "Stagnated", "Worsened"]
pd.cut(violations_evo["Violations evolution"], bins, labels=cats).value_counts().plot.bar()

Most restaurants have improved or stagnated over time in their number of violations. The number of restaurants which got worse over time is smaller than the two other categories.

We will now compare the amount of restaurants who passed their most recent inspection in 2010 and 2018, per area. This will give us some insight in restaurant safety evolution over time.

In [ ]:
# Number of inspections done in 2010
inspections[(inspections['Inspection Date'] >= '2010-01-01') & (inspections['Inspection Date'] < '2011-01-01')]['License #'].nunique()

In [ ]:
# Number of inspections done in 2018
inspections[(inspections['Inspection Date'] >= '2018-01-01') & (inspections['Inspection Date'] < '2019-01-01')]['License #'].nunique()

In [ ]:
# Selecting inspections done in 2010 and 2018
chicago_2010 = inspections[(inspections['Inspection Date'] >= '2010-01-01') & (inspections['Inspection Date'] < '2011-01-01')].copy()
chicago_2018 = inspections[(inspections['Inspection Date'] >= '2018-01-01') & (inspections['Inspection Date'] < '2019-01-01')].copy()
chicago_2010["Results"] = chicago_2010["Results"].apply(lambda x : x.startswith("Pass"))
chicago_2018["Results"] = chicago_2018["Results"].apply(lambda x : x.startswith("Pass"))

In [ ]:
# Computing number of restaurants which passed their last inspection per zip code

pass_by_zip2010 = chicago_2010.groupby(["License #","Zip"])\
                                .apply(lambda g : g.sort_values(by="Inspection Date").iloc[-1].Results)\
                                .groupby("Zip")\
                                .sum()
pass_by_zip2018 = chicago_2018.groupby(["License #","Zip"])\
                                .apply(lambda g : g.sort_values(by="Inspection Date").iloc[-1].Results)\
                                .groupby("Zip")\
                                .sum()

In [ ]:
# Converting counts to int
pass_by_zip2010 = pd.DataFrame({"Zip":pass_by_zip2010.index,"Pass":pass_by_zip2010.values})
pass_by_zip2018 = pd.DataFrame({"Zip":pass_by_zip2018.index,"Pass":pass_by_zip2018.values})
pass_by_zip2010.Pass = pass_by_zip2010.Pass.apply(lambda x : int(x))
pass_by_zip2018.Pass = pass_by_zip2018.Pass.apply(lambda x : int(x))

As not all areas have the same amount of restaurants, for each area we need to divide the number of restaurants passing inspections by the number of restaurants in the area to get a percentage.

In [ ]:
zip_count_2010 = chicago_2010.groupby("Zip").apply(lambda g : g["License #"].nunique())
zip_count_2018 = chicago_2018.groupby("Zip").apply(lambda g : g["License #"].nunique())

In [ ]:
zip_count_2010 = pass_by_zip2010.merge(pd.DataFrame({"Zip":zip_count_2010.index,"Count": zip_count_2010.values}),on='Zip', how='left')
zip_count_2018 = pass_by_zip2018.merge(pd.DataFrame({"Zip":zip_count_2018.index,"Count": zip_count_2018.values}),on='Zip', how='left')

In [ ]:
# Computing pass percentages

zip_count_2010.Pass = zip_count_2010.apply(lambda row : row.Pass/row.Count, axis=1)
zip_count_2018.Pass = zip_count_2018.apply(lambda row : row.Pass/row.Count, axis=1)

In [ ]:
zip_count_2010 = zip_count_2010.drop("Count",axis=1)
zip_count_2018 = zip_count_2018.drop("Count",axis=1)

We can now plot the percentage of valid restaurants per area in 2010...

In [ ]:
state_geo_path = os.path.join(data_folder, "chicago-zip.json")
geo_json_data = json.load(open(state_geo_path))

map_zip_2010 = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
map_zip_2010.choropleth(geo_data=geo_json_data, data=zip_count_2010,
             columns=['Zip', 'Pass'],
             bins = [0.7,0.75,0.8,0.85,0.9,0.95,1.0],
             key_on='feature.properties.zip',
             fill_color='BuGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Percentage of passing restaurants')
map_zip_2010

... and in 2018 :

In [ ]:
state_geo_path = os.path.join(data_folder, "chicago-zip.json")
geo_json_data = json.load(open(state_geo_path))

map_zip_2018 = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
map_zip_2018.choropleth(geo_data=geo_json_data, data=zip_count_2018,
             columns=['Zip', 'Pass'],
             key_on='feature.properties.zip',
             bins = [0.6,0.75,0.8,0.85,0.9,0.95,1.0],
             fill_color='BuGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Percentage of passing restaurants')
map_zip_2018

Overall, it seems that the percentage of restaurants passing inspections has decreased between 2010 and 2018 (which shows that the number of safety violations and and inspection results are not related, as we saw previously that most areas have decreased their number of violations over time). We can notice that the lowest percentage is lower in 2018 compared to 2010.

#### Milestone 3 goals

Our goal for the next milestone is to study more indicators of restaurant quality, such as the risk level and violation categories, and their evolution over time. Particularly, it would be interesting to find which areas show the most potential for future improvement. We could also check whether certain restaurant chains (Subway, Starbucks...) follow certain trends.

### Question 4
**How is restaurant performance in terms of inpection results related to geodemographic charactestics of the area (e.g. Life Quality Index)?**

In this milestone we managed to scrape life quality index per zip code in Chicago, we present first insight as a visualization.

In [ ]:
lqi_zipcode = pd.read_csv(os.path.join(data_folder, "lqi_indexes.csv"), index_col=False)
lqi_zipcode.head()

In [ ]:
# Cast Zip code values to string
lqi_zipcode.zip_code = lqi_zipcode.zip_code.apply(lambda x: str(x))

In [ ]:
lqi_zipcode.nunique()

In [ ]:
inspections.Zip.nunique()

There is one zip code missing in our life quality index, it corresponds to Chicago airport (which makes sense as the airport is not the area where people live). In our analysis we will therefore use it for all of the other neighborhoods.

In [ ]:
# Visualize neighorhoods with respect to the corresponding Life Quality Index
state_geo_path = os.path.join(data_folder, 'chicago-zip.json')
geo_json_data = json.load(open(state_geo_path))

lqi_map = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
lqi_map.choropleth(geo_data=geo_json_data, data=lqi_zipcode,
             columns=['zip_code', 'life_quality_index'],
             key_on='feature.properties.zip',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Life quality index per neighbourhood')
lqi_map

Life quality index in Chicago is the highest in bay area and goes down as we move away, generally we can say that south part of the city has worse life quality than the north part.

#### Milestone 3 goals

Our aim in the next milestone will be to check if there is a relationship between "safe or dangerous" neighborhoods to eat and life quality index in those neighborhoods.


### Question 5
**How are inspection results connected to customer reviews? Are the best scored restaurants the ones with the best inspection results as well? Which client-reported issues are also noticed by inspections? Which issues are only discovered by inspections?**

We decided to use Google Places API to retrieve information about restaurant ratings. We queried API by DBA Name value and Longitude and Latitude of a restaurant. Sometimes however, API responses didn't match the area we were concerned about, meaning that they returned a restaurant with similar name but not in Chicago. Some restaurants couldn't be retrieved at all. To be sure we analyze API response correctly we start by checking if restuarants for which we have ratings are the same ones as we queried for. We perform it by checking names similarity and addresses.

`restaurants.csv` is a file which contains place_id (returned by Google API), restaurant name, address, zip code, latitude and longitude from Chicago Inspections dataset. We will use this csv to compare with what was retrieved from Google API.

In [ ]:
google_places_path = os.path.join(data_folder, "google_places.csv")
restaurants_path = os.path.join(data_folder, "restaurants.csv")

In [ ]:
google_places = pd.read_csv(google_places_path)
restaurants = pd.read_csv(restaurants_path)

In [ ]:
google_places.head()

In [ ]:
restaurants.head()

Let's see if all restaurants we got from Google are the ones we asked for and are located in Chicago:

We noticed that address from Chicago database has whitespace at the end so we remove it from each row:

In [ ]:
restaurants.address = restaurants.address.apply(str.strip)

In [ ]:
print(f"Google places API returned {len(google_places)} restaurants")

Check if all restaurants are in Chicago:

In [ ]:
not_chicago_restaurants = google_places[~(google_places.city == "chicago")]
print(f"Number of restaurants with non-chicago city {len(not_chicago_restaurants)}")

In [ ]:
not_chicago_restaurants.head(10)

We can see that sometimes the zip code was returned as a city, Chicago zip codes start with 60 therefore those entries where city is equal to a number starting with 60 are probably in Chicago and their city name is in the address column

In [ ]:
address_chicago_count = len(google_places[google_places.address == "chicago"])
print(f"Number of places with chicago as address and zip code in city column is {address_chicago_count}")

In [ ]:
google_places[google_places.city.str.startswith("60", na=False)]

If zip code starts with 60 move it to zip code column

In [ ]:
chicago_idx = google_places.city.str.startswith("60", na=False)
google_places.loc[chicago_idx, 'zip_code'] = google_places['city']

In [ ]:
google_places[google_places.city.str.startswith("60", na=False)]

If address is Chicago move it to city column

In [ ]:
address_idx = (google_places.address == "chicago")
google_places.loc[address_idx, "city"] = "chicago"
google_places.loc[address_idx, "address"] = np.nan

In [ ]:
not_chicago_restaurants = google_places[~(google_places.city == "chicago")]
print(f"Number of restaurants with non-chicago city {len(not_chicago_restaurants)}")

We can see that sometimes there are still cities denoted with zip codes starting with 60, let's check city for those values

In [ ]:
google_places[google_places.city.str.startswith("60", na=False)]

It turns out that entries as cicero, north riverside, schaumburg or elgin are within Chicago aglomeration they are smaller towns or districts.

In [ ]:
google_places[(~(google_places.zip_code.str.startswith("60", na=False)) & (google_places.city != "chicago"))]

We have 1071 rows which correspond to restaurants in different places than Chicago area. They are of no use to us so we remove them.

In [ ]:
google_places_cleaned = google_places[((google_places.zip_code.str.startswith("60", na=False)) | (google_places.city == "chicago"))]
google_places_cleaned.head()

We will remove any duplicating rows from both dataframes as sometimes Google API might have returned same place_id for two distinct restaurants

In [ ]:
google_places_cleaned.drop_duplicates(inplace=True)
restaurants.drop_duplicates(inplace=True)

Now we want to compare addresses and names of restaurants. We will start with joining Google places API output dataframe with restaurants dataframe on place_id. We perform left join on `google_places_cleaned` as we are only interested in cases when we received a response from Google API.

In [ ]:
merged = google_places_cleaned.merge(restaurants, on="place_id", how="left", suffixes=("_google", "_restaurants"))
len(merged)

In [ ]:
merged.head()

Sometimes we got a response from Google but we didn't get a rating, we drop rows where we couldn't retrieve rating from Google API.


In [ ]:
merged[merged.rating.isna()]

In [ ]:
merged.dropna(subset=["rating"],axis=0, inplace=True)

To check if for sure we have correct entries we will compare zip codes. If no zip_code is available from restaurants then we will compare names. First step would be to check where `zip_code_restaurants` is NaN
and second would be to remove second zip code component from `zip_code_google`

In [ ]:
merged[merged.zip_code_restaurants.isna()]

We have only a couple of rows where zip_code_restaurants is not available, we can review them manually. It turns out that we will have to remove row 941, 11062 and McCormick place which is not a restaurant but a business center in Chicago.

In [ ]:
merged.drop(axis=0, index=[941, 11062, 11155, 8756], inplace=True)

Now we will remove second `zip_code_google` component

Filling nans with string value to remove second `zip_code_google` component

In [ ]:
merged.zip_code_google.fillna("nan", inplace=True)

In [ ]:
merged.zip_code_google.str.len().unique()

In [ ]:
merged.zip_code_google = merged.zip_code_google.apply(lambda x: x[:5])

In [ ]:
merged.zip_code_google = merged.zip_code_google.replace("nan", np.nan)

In [ ]:
merged.zip_code_google = merged.zip_code_google.apply(float)

Now we will take only those entries where every place has a matching zip code

In [ ]:
matching = merged[merged.zip_code_google == merged.zip_code_restaurants]

At last to be sure that we have rating corresponding to a correct place we will compare names using python difflib. From docs "ratio() returns a float in [0, 1], measuring the similarity of the sequences. As a rule of thumb, a ratio() value over 0.6 means the sequences are close matches". We will use this property to discard any rows where name similarity is lower than 0.6. This step is necessary as sometimes name from Chicago database doesn't exactly match a name returned by Google API for instance in Chicago database we have: "yolk - test kitchen" while Google's response name is "yolk test kitchen"

In [ ]:
from difflib import SequenceMatcher

def get_similarity_score(restaurant_name, google_name):
    return SequenceMatcher(lambda x: x == " ",
                    restaurant_name,
                    google_name).ratio()

In [ ]:
matching['name_similarity_score'] = matching.apply(lambda row: get_similarity_score(row["place_name_restaurants"], row["place_name_google"]), axis=1)

In [ ]:
matching = matching[matching.name_similarity_score > 0.6]

As a last step we remove rows with duplicated place_id

In [ ]:
matching = matching.drop_duplicates(subset=["place_id"])

In [ ]:
len(matching)

we have ratings for 5700 restaurants

We will transform matching dataframe to use it for further analysis, we will remove useless columns and save it to csv file

In [ ]:
matching.drop(["name_similarity_score", "place_name_google", "price_level", "address_google", "zip_code_google", "city"], axis=1, inplace=True)

In [ ]:
matching.rename(columns={"place_name_restaurants": "place_name", "address_restaurants": "address", "zip_code_restaurants": "zip_code"}, inplace=True)

#### Analysis of number of ratings available

In [ ]:
matching.total_number_of_ratings.describe()

In [ ]:
sns.set(rc={'figure.figsize':(12,8)})
sns.distplot(matching.total_number_of_ratings)

In [ ]:
matching.total_number_of_ratings.mode()

As number of ratings seem to follow a power law and most of restaurants have low number of ratings it would be very biased to compare them with restaurants that have tenths of thousands of ratings. Therefore we decide to drop this research question.

## Milestone 3

### Plan of further analysis

Below we present a plan regarding further analysis with respect to particular research questions:

**Question 1:** 
 - visualize and analyze spatial patterns of most common violations, 
 - use violations as a source for "safe to eat" metric for a neighborhood (e.g. with percentage of inspections passed without violations concerning food or a trend in such pass rate).

**Question 2:** 
 - build a better heuristic to evaluate restaurant safety, 
 - use other available features to determine "safe or dangerous" neighborhood (e.g. use the Risk column, narrow down the analysis to certain inspection type).

**Question 3:** 
 - study more indicators of restaurant quality, such as the risk level and violation categories, and their evolution over time,
 - find areas with potential future improvement,
 - inspect whether some restaurant chains follow certain trends.

**Question 4:** 
 - check if there is a relationship between "safe or dangerous" neighborhoods to eat and life quality index in those neighborhoods.


### Data story plan

1. We begin it by general statistics and results based on simple metrics that should bring suspected results (such as the ones we obtained, where the city center and the airport proved to be the safest places to eat).
2. We dive deeper by constructing more complicated metrics based on inspection results and violations.
3. We inspect the trends that can tell us more about how neighborhoods are evolving. It *can* allow for forecasting which neighborhoods might be promising in the near future. This analysis can focus also on certain interesting restaurant chains.
4. We conclude our data story with recommendations of neighborhoods where one may be more interested to eat based on the previously presented metrics and analysis.

Throught our data story we want to make an extensive use of maps to visualize our metrics.

The final presentation will follow the pattern similar to the data story but limited to the most relevant/interesting information and findings.